# Глава 10 Агрегирование данных и групповые опперации

## 10.1. Механизм Group By

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame({'key1':['a','a','b','b','a'], 'key2':['one','two','one','two','one'], 'data1':np.random.randn(5),'data2':np.random.randn(5)})

In [5]:
df

,key1,key2,data1,data2
0,a,one,-0.704549,1.404107
1,a,two,0.311428,-0.259695
2,b,one,0.264342,-0.120834
3,b,two,0.254318,0.117681
4,a,one,0.382368,0.272459


In [8]:
grouped = df['data1'].groupby(df['key1'])
grouped 

In [9]:
grouped.mean()

key1
a   -0.003584
b    0.259330
Name: data1, dtype: float64

In [11]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()

In [12]:
means

key1  key2
a     one    -0.161091
      two     0.311428
b     one     0.264342
      two     0.254318
Name: data1, dtype: float64

In [13]:
means.unstack()

key2,one,two
key1,,
a,-0.161091,0.311428
b,0.264342,0.254318


In [15]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])

In [16]:
df['data1'].groupby([states,years]).mean()

California  2005    0.311428
            2006    0.264342
Ohio        2005   -0.225115
            2006    0.382368
Name: data1, dtype: float64

In [17]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.003584,0.472290
b,0.259330,-0.001577


In [18]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.161091  0.838283
     two   0.311428 -0.259695
b    one   0.264342 -0.120834
     two   0.254318  0.117681

In [19]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Обход групп

In [25]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)
    

a
  key1 key2     data1     data2
0    a  one -0.704549  1.404107
1    a  two  0.311428 -0.259695
4    a  one  0.382368  0.272459
b
  key1 key2     data1     data2
2    b  one  0.264342 -0.120834
3    b  two  0.254318  0.117681


In [27]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print(k1,k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one -0.704549  1.404107
4    a  one  0.382368  0.272459
a two
  key1 key2     data1     data2
1    a  two  0.311428 -0.259695
b one
  key1 key2     data1     data2
2    b  one  0.264342 -0.120834
b two
  key1 key2     data1     data2
3    b  two  0.254318  0.117681


In [28]:
pieces = dict(list(df.groupby('key1')))

In [29]:
pieces

{'a':   key1 key2     data1     data2
 0    a  one -0.704549  1.404107
 1    a  two  0.311428 -0.259695
 4    a  one  0.382368  0.272459,
 'b':   key1 key2     data1     data2
 2    b  one  0.264342 -0.120834
 3    b  two  0.254318  0.117681}

In [41]:
df

,key1,key2,data1,data2
0,a,one,-0.704549,1.404107
1,a,two,0.311428,-0.259695
2,b,one,0.264342,-0.120834
3,b,two,0.254318,0.117681
4,a,one,0.382368,0.272459


In [42]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.838283
     two  -0.259695
b    one  -0.120834
     two   0.117681

In [44]:
s_grouped = df.groupby(['key1','key2'])['data2']
s_grouped

In [45]:
s_grouped.mean()

key1  key2
a     one     0.838283
      two    -0.259695
b     one    -0.120834
      two     0.117681
Name: data2, dtype: float64

### Группировка с  помощью словарей и  объектов Series

In [48]:
people = pd.DataFrame(np.random.randn(5,5,), columns = ['a','b','c','d','e'], index = ['Joe','Steve','Wes','Jim','Travis'])

In [49]:
people

,a,b,c,d,e
Joe,0.931936,-0.303941,-0.061386,-1.226331,-1.460279
Steve,-0.956158,-1.842928,0.190926,-1.504697,0.032894
Wes,-2.370797,-0.880861,0.690830,0.944466,1.389931
Jim,1.422736,0.265097,-0.249010,-0.504490,-1.218527
Travis,-1.437912,-0.050297,0.867066,0.933183,-0.774523


In [52]:
people.loc[2:3,['b','c']] = np.nan

<ipython-input-52-de208cb63c41>:1: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  people.loc[2:3,['b','c']] = np.nan


In [53]:
people

,a,b,c,d,e
Joe,0.931936,-0.303941,-0.061386,-1.226331,-1.460279
Steve,-0.956158,-1.842928,0.190926,-1.504697,0.032894
Wes,-2.370797,NaN,NaN,0.944466,1.389931
Jim,1.422736,0.265097,-0.249010,-0.504490,-1.218527
Travis,-1.437912,-0.050297,0.867066,0.933183,-0.774523


In [54]:
mapping = {'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}

In [55]:
by_column = people.groupby(mapping,axis = 1)

In [57]:
by_column.sum()

,blue,red
Joe,-1.287717,-0.832284
Steve,-1.313772,-2.766193
Wes,0.944466,-0.980866
Jim,-0.753500,0.469306
Travis,1.800249,-2.262732


In [60]:
map_series = pd.Series(mapping)

In [61]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [62]:
people.groupby(map_series,axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Группировка с  помощью функций

In [63]:
people

,a,b,c,d,e
Joe,0.931936,-0.303941,-0.061386,-1.226331,-1.460279
Steve,-0.956158,-1.842928,0.190926,-1.504697,0.032894
Wes,-2.370797,NaN,NaN,0.944466,1.389931
Jim,1.422736,0.265097,-0.249010,-0.504490,-1.218527
Travis,-1.437912,-0.050297,0.867066,0.933183,-0.774523


In [64]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.016124,-0.038844,-0.310396,-0.786354,-1.288876
5,-0.956158,-1.842928,0.190926,-1.504697,0.032894
6,-1.437912,-0.050297,0.867066,0.933183,-0.774523


In [66]:
key_list = ['one','one','one','two','two']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -2.370797 -0.303941 -0.061386 -1.226331 -1.460279
  two  1.422736  0.265097 -0.249010 -0.504490 -1.218527
5 one -0.956158 -1.842928  0.190926 -1.504697  0.032894
6 two -1.437912 -0.050297  0.867066  0.933183 -0.774523

### Группировка по уровням индекса

In [68]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]], names = ['cty','tenor'])

In [71]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns = columns)

In [72]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.640623  1.130751  0.662294 -0.129327  1.925209
1      0.420720 -0.678976  0.793113 -0.819099  2.310258
2      0.420964  1.209381 -0.443695 -0.876749 -0.210933
3     -0.042987  0.028194 -0.792311 -0.916282 -1.499247

In [73]:
hier_df.groupby(level = 'cty', axis =1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


### 10.2. Агрегирование данных

In [75]:
df

,key1,key2,data1,data2
0,a,one,-0.704549,1.404107
1,a,two,0.311428,-0.259695
2,b,one,0.264342,-0.120834
3,b,two,0.254318,0.117681
4,a,one,0.382368,0.272459


In [76]:
grouped = df.groupby('key1')

In [77]:
grouped['data1'].quantile(0.9)

key1
a    0.36818
b    0.26334
Name: data1, dtype: float64

In [78]:
def peak_to_peak(arr):
    return arr.max() - arr.min()


In [79]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.086916,1.663801
b,0.010024,0.238515


In [80]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.003584  0.608088 -0.704549 -0.196561  0.311428  0.346898   
b      2.0  0.259330  0.007088  0.254318  0.256824  0.259330  0.261836   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.382368   3.0  0.472290  0.849711 -0.259695  0.006382  0.272459   
b     0.264342   2.0 -0.001577  0.168656 -0.120834 -0.061206 -0.001577   

                          
           75%       max  
key1                      
a     0.838283  1.404107  
b     0.058052  0.117681